# Experiments [1.1.1] [1.1.2] - Rationales Frequencies

In [1]:
def param_default():
    return {
        'model': 'codeparrot',
        'datasets' : {
            'SG_BD' : 'code_completion_random_cut_5k_30_512_tokens', 
            'DC_SG_BD' : 'code_completion_docstring_random_cut_3.8k_30_150_tokens', 
            'DC_SG' : 'code_completion_docstring_signature_3.8k_30_150_tokens', 
            'DC': 'code_completion_docstring_5k_30_150_tokens'
        },
        'modality' : 'sc',
        ######## INPUT 
        'rational_results': '/workspaces/code-rationales/data/rationales/gpt',
        'sampling_results': '/workspaces/code-rationales/data/sampling/gpt',
        'tagged_rationales': '/workspaces/code-rationales/data/tagged_rationales/pure_code',
        'global_ast_results': '/workspaces/code-rationales/data/global_ast_results/gpt/pure_code',
        'grouping_results': '/workspaces/code-rationales/data/experiments/grouping_results', 
        ######## OUTPUT
        'rationales_distributions': '/workspaces/code-rationales/data/experiments/rationales_distributions',
        'num_experiments': 30,
        'num_samples': 100,
    }
    
params = param_default()

## Source Code

In [2]:
from pathlib import Path
import csv
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools
import json
import os.path
from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = '{:.2f}'.format

In [6]:
get_experiment_path =  lambda results_folder, dataset, exp: results_folder + '/' + dataset + '_exp_' + str(exp) +'.txt'

In [7]:
def load_ast_rationales_results(dataset_id: str) :
    experiment_paths = [get_experiment_path(params['global_ast_results'], params['datasets'][dataset_id], exp) for exp in range(params['num_experiments'])]
    experiment_results = []
    for experiment_path in experiment_paths:
        with open(experiment_path, 'r') as file:
            experiment_results.append(json.loads(file.read()))
    return experiment_results

In [9]:
rationales_results = load_ast_rationales_results('SG_BD')

In [5]:
rationales_results

[             goal_token  from_seq_id  \
 rational_pos                           
 0                   def            0   
 1                  skip            0   
 2                     _            0   
 3                    on            0   
 4                     _            0   
 ...                 ...          ...   
 39                   ('           99   
 40                  pip           99   
 41                   ',           99   
 42                    '           99   
 43                 auto           99   
 
                                                    typesets_tgt  \
 rational_pos                                                      
 0                                                           NaN   
 1                             [('def', 0.00029721998726017773)]   
 2                               [(' skip', 0.3496493101119995)]   
 3             [('_', 0.001134732156060636), (' skip', 0.0059...   
 4                                  [('on', 0.22727505862